In [5]:
# laser_triangulation_clean.py
# -*- coding: utf-8 -*-
%matplotlib notebook

import os
import glob
import numpy as np
import cv2
from scipy import ndimage
import matplotlib.pyplot as plt

# =======================
# User-Parameter
# =======================
# 1) Bildquellen (nimm einfach zwei Dateien in beliebiger Reihenfolge)
IMG_DIR = r"G:\Meine Ablage\Studium\Master\3. Semester\Masterprojekt\Kamera_Simulationen\Solidworks"
IMG_PATTERNS = ["**/*.png", "**/*.jpg", "**/*.jpeg"]

# 2) Ziel-Aspect-Ratio (Breite:Höhe) – nur benutzen, wenn SolidWorks falsch speichert
USE_CROP = True
TARGET_AR = (8.5, 11.0)  # width : height

# 3) Intrinsics: Entweder HFoV ODER (f und Sensor)
USE_FOV = True
HFOV_DEG = 39.6  # 50mm Vollformat -> ~39.6° horizontal
# Falls du lieber mit Sensor+f rechnest:
F_MM = 50.0
SENSOR_W_MM = 36.0
SENSOR_H_MM = 24.0

# 4) Extrinsics: Kamera-Zentren in Weltkoordinaten (mm) + „look-at Ursprung“
VC1 = np.array([+25.0, 53.59, 200.0])
VC2 = np.array([-25.0, 53.59, 200.0])

# 5) Bild-/Kameraachsen-Konvention:
# Bild-v zeigt nach unten; für klassische Kamera-Y-nach-oben kannst du flippen:
FLIP_IMAGE_Y_TO_CAM = True  # d.h. nach K^{-1} wird d_cam[1] *= -1

# =======================
# Helper
# =======================

def list_images(base, patterns):
    ims = []
    for p in patterns:
        ims.extend(glob.glob(os.path.join(base, p), recursive=True))
    ims = [f for f in ims if os.path.isfile(f)]
    ims.sort()
    if len(ims) < 2:
        raise FileNotFoundError("Mindestens zwei Bilder werden benötigt.")
    return ims[:2]

def load_rgb(path):
    bgr = cv2.imread(path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise IOError(f"Bild konnte nicht gelesen werden: {path}")
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def center_crop_to_ar(img, ar_wh):
    """Center-Crop auf gewünschte Aspect-Ratio. Gibt Bild + (x0,y0)-Offset zurück."""
    h, w = img.shape[:2]        # ACHTUNG: (H, W, C)
    target = ar_wh[0] / ar_wh[1]
    if (w / h) > target:
        # zu breit -> Breite kürzen
        new_w = int(round(h * target))
        x0 = (w - new_w) // 2
        y0 = 0
        x1 = x0 + new_w
        y1 = h
    else:
        # zu hoch -> Höhe kürzen
        new_h = int(round(w / target))
        x0 = 0
        y0 = (h - new_h) // 2
        x1 = w
        y1 = y0 + new_h
    return img[y0:y1, x0:x1].copy(), (x0, y0)

def K_from_fovy(image_shape, fov_y_deg, aspect_ratio=None):
    h, w = image_shape[:2]
    fov_y = np.deg2rad(fov_y_deg)
    fy = h / (2.0 * np.tan(fov_y / 2.0))

    if aspect_ratio is None:
        aspect_ratio = w / h
    fov_x = 2.0 * np.arctan(aspect_ratio * np.tan(fov_y / 2.0))
    fx = w / (2.0 * np.tan(fov_x / 2.0))

    cx, cy = w / 2.0, h / 2.0
    return np.array([[fx, 0,  cx],
                     [0,  fy, cy],
                     [0,   0,  1.0]], float)


def K_from_sensor_f(image_shape, f_mm, sensor_w_mm, sensor_h_mm):
    """Intrinsics aus f und Sensor (phys.) + Bildgröße."""
    h, w = image_shape[:2]
    fx = (w / sensor_w_mm) * f_mm
    fy = (h / sensor_h_mm) * f_mm
    cx = w / 2.0
    cy = h / 2.0
    return np.array([[fx, 0,  cx],
                     [0,  fy, cy],
                     [0,   0,  1.0]], float)

def adjust_K_after_crop(K, crop_offset_xy):
    """Verschiebt den Hauptpunkt nach Center-Crop (nur Offsets abziehen)."""
    x0, y0 = crop_offset_xy
    K_adj = K.copy()
    K_adj[0, 2] -= x0
    K_adj[1, 2] -= y0
    return K_adj

def find_red_centroid(img_rgb):
    """Sehr einfache Rot-Maske + Schwerpunkt (in Pixeln: v=row, u=col)."""
    # Schwellen robust gegen „kräftiges“ Rot (anpassen falls nötig)
    lower = np.array([100, 0, 0], dtype=np.uint8)   # RGB min
    upper = np.array([255, 80, 80], dtype=np.uint8) # RGB max
    mask = cv2.inRange(img_rgb, lower, upper)
    if mask.max() == 0:
        raise ValueError("Keine roten Pixel gefunden – Maske anpassen.")
    com = ndimage.center_of_mass(mask)  # (v,row,  u,col) in float
    v, u = float(com[0]), float(com[1])
    return (v, u), mask

def look_at_R(camera_center, target=np.zeros(3), up_hint=np.array([0, 0, 1.0])):
    """
    Baut R_wc (Spalten = x_c, y_c, z_c im Welt-KS), Kamera blickt zum target.
    Roll ist damit durch up_hint festgelegt (falls möglich).
    """
    c = np.asarray(camera_center, float)
    zc = (target - c)
    zc = zc / np.linalg.norm(zc)

    xc = np.cross(up_hint, zc)
    if np.linalg.norm(xc) < 1e-9:
        # up_hint nahezu parallel -> wähle alternative up
        up_hint = np.array([0, 1.0, 0], float)
        xc = np.cross(up_hint, zc)
    xc = xc / np.linalg.norm(xc)
    yc = np.cross(zc, xc)

    R = np.column_stack((xc, yc, zc))  # 3x3
    # Sicherstellen, dass det(R) ~ 1
    if np.linalg.det(R) < 0:
        R[:, 1] *= -1.0  # Spiegel korrigieren
    return R

def pixel_ray_world(u, v, K, R_wc, flip_image_y):
    """Einheitsstrahlrichtung in Weltkoordinaten aus Pixel (u,v)."""
    d_cam = np.linalg.inv(K) @ np.array([u, v, 1.0], float)
    if flip_image_y:
        d_cam[1] *= -1.0
    d_cam /= np.linalg.norm(d_cam)
    d_world = R_wc @ d_cam
    d_world /= np.linalg.norm(d_world)
    return d_world

def unit(v):
    v = np.asarray(v, float)
    n = np.linalg.norm(v)
    return v if n == 0 else v / n

# =======================
# Hauptablauf
# =======================

def main():
    # ---- Bilder laden
    img_paths = list_images(IMG_DIR, IMG_PATTERNS)
    img1 = load_rgb(img_paths[0])
    img2 = load_rgb(img_paths[1])

    # ---- Intrinsics vor Crop
    if USE_FOV:
        K1 = K_from_fovy(img1.shape, fov_y_deg=26.99, aspect_ratio=11/8.5)

        K2 = K_from_fovy(img2.shape, fov_y_deg=26.99, aspect_ratio=11/8.5)

    else:
        K1 = K_from_sensor_f(img1.shape, F_MM, SENSOR_W_MM, SENSOR_H_MM)
        K2 = K_from_sensor_f(img2.shape, F_MM, SENSOR_W_MM, SENSOR_H_MM)

    # ---- Optionaler Center-Crop + K anpassen
    off1 = (0, 0)
    off2 = (0, 0)
    if USE_CROP:
        img1, off1 = center_crop_to_ar(img1, TARGET_AR)
        img2, off2 = center_crop_to_ar(img2, TARGET_AR)
        K1 = adjust_K_after_crop(K1, off1)
        K2 = adjust_K_after_crop(K2, off2)

    # ---- Rote Schwerpunkte finden (v,u!)
    (v1, u1), mask1 = find_red_centroid(img1)
    (v2, u2), mask2 = find_red_centroid(img2)

    # ---- Rotationsmatrizen (Kameras schauen zum Ursprung)
    R1 = look_at_R(VC1)
    R2 = look_at_R(VC2)

    # ---- Strahlenrichtungen in Weltkoordinaten
    d1_w = pixel_ray_world(u1, v1, K1, R1, FLIP_IMAGE_Y_TO_CAM)
    d2_w = pixel_ray_world(u2, v2, K2, R2, FLIP_IMAGE_Y_TO_CAM)

    # ---- „Original“-Richtungen (zum Ursprung) – zum Vergleich
    r1_to_origin = unit(-VC1)
    r2_to_origin = unit(-VC2)

    # =======================
    # Plot
    # =======================
    fig = plt.figure(figsize=(9, 8))
    ax = fig.add_subplot(111, projection='3d')

    L = 300.0
    t = np.linspace(0.0, 1.0, 2)

    # Kamerazentren
    ax.scatter([VC1[0]], [VC1[1]], [VC1[2]], s=40, label="VC1")
    ax.scatter([VC2[0]], [VC2[1]], [VC2[2]], s=40, label="VC2")
    ax.scatter([0], [0], [0], s=40, label="World Origin")
    ax.scatter([-10], [25], [25], s=40, label="Laser Punkt")  # optionaler Referenzpunkt

    # Ray → Ursprung (dotted)
    P1 = VC1 + np.outer(t * L, r1_to_origin)
    P2 = VC2 + np.outer(t * L, r2_to_origin)
    ax.plot(P1[:, 0], P1[:, 1], P1[:, 2], linestyle="dotted", alpha=0.5, label="VC1 → origin")
    ax.plot(P2[:, 0], P2[:, 1], P2[:, 2], linestyle="dotted", alpha=0.5, label="VC2 → origin")

    # Gemessene Pixel-Strahlen
    Q1 = VC1 + np.outer(t * L, d1_w)
    Q2 = VC2 + np.outer(t * L, d2_w)
    ax.plot(Q1[:, 0], Q1[:, 1], Q1[:, 2], label="VC1 pixel-ray")
    ax.plot(Q2[:, 0], Q2[:, 1], Q2[:, 2], label="VC2 pixel-ray")

    # Kleiner Würfel (zur Orientierung)
    x = [-25, 25]; y = [0, 50]; z = [-25, 25]
    cube = np.array([[xi, yi, zi] for xi in x for yi in y for zi in z])
    edges = [(0,1),(0,2),(0,4),(3,1),(3,2),(3,7),(5,1),(5,4),(5,7),(6,2),(6,4),(6,7),(3,6)]
    for i,j in edges:
        ax.plot([cube[i,0], cube[j,0]], [cube[i,1], cube[j,1]], [cube[i,2], cube[j,2]], color="gray", alpha=0.7)

    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    ax.legend(loc="best")

    # „Equal-ish“ Aspect
    allp = np.vstack([P1, P2, Q1, Q2, VC1[None,:], VC2[None,:], np.zeros((1,3))])
    c = (allp.max(0) + allp.min(0)) / 2.0
    e = (allp.max(0) - allp.min(0)).max() / 2.0 + 1e-6
    ax.set_xlim(c[0]-e, c[0]+e); ax.set_ylim(c[1]-e, c[1]+e); ax.set_zlim(c[2]-e, c[2]+e)
    ax.view_init(elev=90, azim=-90)  # gleiche Draufsicht wie im Notebook

    plt.tight_layout()
    plt.show()

    # Debug-Ausgabe
    print(f"Bild1 Größe nach Crop: {img1.shape[:2]}  Offset: {off1}  K1:\n{K1}")
    print(f"Bild2 Größe nach Crop: {img2.shape[:2]}  Offset: {off2}  K2:\n{K2}")
    print(f"COM1 (v,u): {(v1,u1)}   COM2 (v,u): {(v2,u2)}")
    print(f"Richtungen Welt:\n  d1={d1_w}\n  d2={d2_w}")

if __name__ == "__main__":
    main()


<IPython.core.display.Javascript object>

Bild1 Größe nach Crop: (1760, 1360)  Offset: (1240, 0)  K1:
[[6.18216684e+03 0.00000000e+00 6.80000000e+02]
 [0.00000000e+00 3.66687347e+03 8.80000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Bild2 Größe nach Crop: (1760, 1360)  Offset: (1240, 0)  K2:
[[6.18216684e+03 0.00000000e+00 6.80000000e+02]
 [0.00000000e+00 3.66687347e+03 8.80000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
COM1 (v,u): (1011.6323038397329, 428.72328881469116)   COM2 (v,u): (1013.2235576923077, 543.0861378205128)
Richtungen Welt:
  d1=[-0.14194256 -0.20826729 -0.96771744]
  d2=[ 0.0849937  -0.23453022 -0.9683861 ]


In [ ]:
# -*- coding: utf-8 -*-
%matplotlib notebook
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage

# -----------------------------
# Konfiguration
# -----------------------------
BASE_DIR = r"G:\Meine Ablage\Studium\Master\3. Semester\Masterprojekt\Kamera_Simulationen\Solidworks\blick auf ursprung"
ASPECT = (8.5, 11)        # (width, height)
FOV_X_DEG = 26.99         # horizontaler FoV in Grad (falls dein 26.99° vertikal ist -> Hinweis in compute_K_from_fovx beachten)
LOOK_AT = np.array([0.0, 25.0, 0.0])  # Kameras schauen auf (0,25,0)
UP_VEC  = np.array([0.0, 1.0, 0.0])   # global "oben" = +Y
VC1 = np.array([ 25.0, 53.59 + 25, 200.0]) # Kamerazentrum 1
VC2 = np.array([-25.0, 53.59 + 25, 200.0]) # Kamerazentrum 2
LASER_POINT = np.array([-10.0, 25.0, 25.0])  # optionaler Referenzpunkt aus der Simulation (zum Plot)

# Roter Bereich (RGB)
LOWER_RED = np.array([100,   0,   0], dtype=np.uint8)
UPPER_RED = np.array([255,  80,  80], dtype=np.uint8)

# -----------------------------
# Hilfsfunktionen Geometrie
# -----------------------------
def croptoaspect(img, ar):
    """Center-Crop auf Ziel-Seitenverhältnis ar=(width,height)."""
    h, w, _ = img.shape
    target_ratio = ar[0] / ar[1]  # width/height
    if (w / h) > target_ratio:
        # zu breit -> Breite kürzen
        new_w = int(h * target_ratio)
        x0 = (w - new_w) // 2; x1 = x0 + new_w
        y0, y1 = 0, h
    else:
        # zu hoch -> Höhe kürzen
        new_h = int(w / target_ratio)
        y0 = (h - new_h) // 2; y1 = y0 + new_h
        x0, x1 = 0, w
    return img[y0:y1, x0:x1]

def compute_K_from_fovx(W, H, fov_x_deg):
    """
    K aus horizontalem FoV und Bildgröße.
    Falls dein gegebener FoV vertikal ist: tausche Rollen von (W,H) und fov_x_deg <-> fov_y_deg analog.
    """
    fov_x = np.deg2rad(fov_x_deg)
    # vertikaler FoV aus Aspect Ratio
    fov_y = 2.0 * np.arctan((H / W) * np.tan(fov_x / 2.0))

    fx = W / (2.0 * np.tan(fov_x / 2.0))
    fy = H / (2.0 * np.tan(fov_y / 2.0))
    cx, cy = W / 2.0, H / 2.0

    K = np.array([[fx, 0.0, cx],
                  [0.0, fy, cy],
                  [0.0, 0.0, 1.0]], dtype=float)
    return K

def getR(C, look_at=LOOK_AT, up=UP_VEC):
    """
    Rotationsmatrix R_wc (Spalten = Kameraachsen im Welt-KS),
    Kamera schaut von Zentrum C auf look_at; up fixiert den Rollwinkel.
    """
    zc = (look_at - C).astype(float)
    zc /= np.linalg.norm(zc)
    xc = np.cross(up, zc); xc /= np.linalg.norm(xc)
    yc = np.cross(zc, xc)
    R_wc = np.column_stack((xc, yc, zc))
    return R_wc

def pixel_to_camera_ray(u, v, K):
    """liefert normierten Richtungsvektor im Kamera-KS: d_cam ∝ K^{-1}[u,v,1]."""
    sv = np.array([float(u), float(v), 1.0])
    d_cam = np.linalg.inv(K) @ sv
    d_cam /= np.linalg.norm(d_cam)
    return d_cam

def pixel_to_world_ray(com, K, R_wc, C):
    """
    Aus Pixel-Schwerpunkt (v,u) -> Weltstrahl (origin=C, direction=d_world).
    center_of_mass liefert (v,u) = (row,col) -> erst auf (u,v) drehen!
    """
    v, u = float(com[0]), float(com[1])
    d_cam = pixel_to_camera_ray(u, v, K)
    d_world = R_wc @ d_cam
    d_world /= np.linalg.norm(d_world)
    return C.astype(float), d_world

# -----------------------------
# Plot-Helper
# -----------------------------
def set_axes_equal(ax):
    """Gleiche Skalierung für alle 3 Achsen (würfelförmiger Ausschnitt)."""
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()
    x_range = abs(x_limits[1] - x_limits[0])
    y_range = abs(y_limits[1] - y_limits[0])
    z_range = abs(z_limits[1] - z_limits[0])
    max_range = max([x_range, y_range, z_range]) / 2.0
    x_middle = np.mean(x_limits); y_middle = np.mean(y_limits); z_middle = np.mean(z_limits)
    ax.set_xlim3d([x_middle - max_range, x_middle + max_range])
    ax.set_ylim3d([y_middle - max_range, y_middle + max_range])
    ax.set_zlim3d([z_middle - max_range, z_middle + max_range])

def plot_ray(ax, base, direction, scale=300, label=None, color=None, ls='--', alpha=0.9):
    base = np.asarray(base, float)
    d = np.asarray(direction, float)
    n = np.linalg.norm(d);
    if n == 0: return
    d = d / n
    pts = np.vstack([base, base + scale * d])
    ax.plot(pts[:,0], pts[:,1], pts[:,2], label=label, color=color, linestyle=ls, alpha=alpha)

def plotvec_from_origin(ax, v, label=None, color=None, ls='-'):
    v = np.asarray(v, float)
    ax.plot((0, v[0]), (0, v[1]), (0, v[2]), label=label, color=color, linestyle=ls)

def plotcube(ax):
    # Quader um den Ursprung: X∈[-25,25], Y∈[0,50], Z∈[-25,25]
    x = [-25, 25]; y = [0, 50]; z = [-25, 25]
    cube_points = np.array([[xi, yi, zi] for xi in x for yi in y for zi in z], float)
    edges = [(0,1), (0,2), (0,4), (3,1), (3,2), (3,7),
             (5,1), (5,4), (5,7), (6,2), (6,4), (6,7), (3,6)]
    for i,j in edges:
        ax.plot([cube_points[i,0], cube_points[j,0]],
                [cube_points[i,1], cube_points[j,1]],
                [cube_points[i,2], cube_points[j,2]],
                color="gray", linewidth=1)

# -----------------------------
# Bilder suchen & laden
# -----------------------------
patterns = [os.path.join(BASE_DIR, "**", ext) for ext in ("*.jpg","*.jpeg","*.png")]
images = []
for pat in patterns:
    images.extend(glob.glob(pat, recursive=True))
images = sorted(images)
if len(images) < 2:
    raise RuntimeError("Weniger als 2 Bilder gefunden.")

img1 = cv2.cvtColor(cv2.imread(images[0]), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(images[1]), cv2.COLOR_BGR2RGB)

# Crop auf Aspect
img1_c = croptoaspect(img1, ASPECT)
img2_c = croptoaspect(img2, ASPECT)

H, W = img1_c.shape[:2]

# -----------------------------
# Rote Pixel -> Schwerpunkt
# -----------------------------
mask1 = cv2.inRange(img1_c, LOWER_RED, UPPER_RED)
mask2 = cv2.inRange(img2_c, LOWER_RED, UPPER_RED)
com1 = ndimage.center_of_mass(mask1)  # (v,u)
com2 = ndimage.center_of_mass(mask2)  # (v,u)

print("Bildgröße (W,H):", W, H)
print("COM1 (v,u):", com1, " | COM2 (v,u):", com2)

# optional kontroll-Plot Maske + Kreuz
cm = np.bitwise_or(mask1, mask2).copy()
v1, u1 = map(int, com1); v2, u2 = map(int, com2)
cv2.drawMarker(cm, (u1, v1), 255, markerType=cv2.MARKER_CROSS, markerSize=21, thickness=2)
cv2.drawMarker(cm, (u2, v2), 255, markerType=cv2.MARKER_CROSS, markerSize=21, thickness=2)
# plt.imshow(cm, cmap='gray', vmin=0, vmax=255); plt.show()

# -----------------------------
# K, R, Strahlen
# -----------------------------
K = compute_K_from_fovx(W, H, FOV_X_DEG)
R1 = getR(VC1, LOOK_AT, UP_VEC)
R2 = getR(VC2, LOOK_AT, UP_VEC)

origin1, d_world1 = pixel_to_world_ray(com1, K, R1, VC1)
origin2, d_world2 = pixel_to_world_ray(com2, K, R2, VC2)

# optional: Check Winkel zum Laserpunkt
def angle_to_point(origin, d, P):
    v = (P - origin); v /= np.linalg.norm(v)
    return np.degrees(np.arccos(np.clip(np.dot(d, v), -1, 1)))
ang1 = angle_to_point(origin1, d_world1, LASER_POINT)
ang2 = angle_to_point(origin2, d_world2, LASER_POINT)
print(f"Winkel Cam1-Ray -> Laserpunkt: {ang1:.3f}°")
print(f"Winkel Cam2-Ray -> Laserpunkt: {ang2:.3f}°")

# -----------------------------
# Plot
# -----------------------------
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111, projection="3d")
ax.set_box_aspect([1,1,1])

# Kamerazentren
ax.scatter([VC1[0]], [VC1[1]], [VC1[2]], s=40, label="VC1")
ax.scatter([VC2[0]], [VC2[1]], [VC2[2]], s=40, label="VC2")
ax.scatter([0], [0], [0], s=30, label="World Origin")
ax.scatter([LOOK_AT[0]],[LOOK_AT[1]],[LOOK_AT[2]], s=30, label="look_at")

# Ursprungswürfel
plotcube(ax)

# Grobe Richtungen der Kamerazentren von Ursprung aus (nur zur Orientierung)
plotvec_from_origin(ax, VC1, label="VC1 (vom Ursprung)", color="C2")
plotvec_from_origin(ax, VC2, label="VC2 (vom Ursprung)", color="C3")

# Strahlen (aus Pixel-Schwerpunkten)
plot_ray(ax, origin1, d_world1, scale=300, label="Cam1 Ray", color="C0", ls="--", alpha=0.8)
plot_ray(ax, origin2, d_world2, scale=300, label="Cam2 Ray", color="C1", ls="--", alpha=0.8)

# Optional: Laserpunkt
ax.scatter([LASER_POINT[0]], [LASER_POINT[1]], [LASER_POINT[2]],
           color='red', s=30, label="Laser Punkt")

# Achsen & Ansicht
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.view_init(elev=90, azim=-90)  # XY-Ansicht (Z nach oben). Passe bei Bedarf an.
ax.legend(loc="best")
set_axes_equal(ax)
plt.tight_layout()
plt.show()
